# Part 3: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [1]:
import syft as sy
hook = sy.TorchHook()

# Section 3.1 - Pointers to Pointers

As you know, PointerTensor objects feel just like normal tensors. In fact, they are _so much like tensors_ that we can even have pointers TO the pointers. Check it out!

In [2]:
bob = sy.VirtualWorker(id='bob')
alice = sy.VirtualWorker(id='alice')

# making sure that bob/alice know about each other
bob.add_worker(alice)
alice.add_worker(bob)

In [3]:
# this is a local tensor
x = sy.FloatTensor([1,2,3,4])
x


 1
 2
 3
 4
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

In [4]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

FloatTensor[_PointerTensor - id:97567697271 owner:me loc:bob id@loc:59864775026]

In [5]:
x_ptr.child.original_pointer

True

In [6]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

FloatTensor[_PointerTensor - id:97567697271 owner:me loc:alice id@loc:39298987248]

### What happened?

So, in the previous example, we created a tensor called "x" and send it to Bob, creating a pointer on our local machine ("x_ptr"). 

Then, we called x_ptr.send(alice) which SENT THE POINTER to Alice. 

Note, this did NOT move the data! Instead, it moved the pointer to the data!! 

In [7]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

{59864775026: [_LocalTensor - id:59864775026 owner:bob]}

In [8]:
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
alice._objects

{39298987248: [_PointerTensor - id:39298987248 owner:alice loc:bob id@loc:59864775026]}

In [9]:
# and we can use .get() to get x_ptr back from Alice

x_ptr = pointer_to_x_ptr.get()
x_ptr

FloatTensor[_PointerTensor - id:97567697271 owner:me loc:bob id@loc:59864775026]

In [10]:
# and then we can use x_ptr to get x back from Bob!

x = x_ptr.get()
x


 1
 2
 3
 4
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

### Arithmetic on Pointer -> Pointer -> Data Object

And just like with normal pointers, we can perform arbitrary PyTorch operations across these tensors

In [11]:
bob._objects

{}

In [12]:
alice._objects

{}

In [13]:
p2p2x = sy.FloatTensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [14]:
bob._objects

{71136639287: [_LocalTensor - id:71136639287 owner:bob],
 83916690689: [_LocalTensor - id:83916690689 owner:bob]}

In [15]:
alice._objects

{15542574535: [_PointerTensor - id:15542574535 owner:alice loc:bob id@loc:71136639287],
 41676431577: [_PointerTensor - id:41676431577 owner:alice loc:bob id@loc:83916690689]}

In [16]:
y.get().get()


  2
  4
  6
  8
 10
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [17]:
bob._objects

{83916690689: [_LocalTensor - id:83916690689 owner:bob]}

In [18]:
alice._objects

{41676431577: [_PointerTensor - id:41676431577 owner:alice loc:bob id@loc:83916690689]}

In [19]:
p2p2x.get().get()


 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [20]:
bob._objects

{}

In [21]:
alice._objects

{}

# Section 3.2 - Pointer Chain Operations

So in the last section whenever we called a .send() or a .get() operation, it called that operation directly on the tensor on our local machine. However, if you have a chain of pointers, sometimes you want to call operations like .get() or .send() on the LAST pointer in the chain (such as sending data directly from one worker to another). To accomplish this, you want to use functions which are especially designed for this privacy preserving operation.

These operations are:

- my_poitner2pointer.end_get()
- my_pointer2pointer.move(another_worker)

Let's start with .end_get(). This one simply identifies the _last_ pointer in the chain and calls .get() on that pointer! It's an inline operation. Let's look at an example.

In [22]:
# x is now a pointer to a pointer to the data which lives on Bob's machine
x = sy.FloatTensor([1,2,3,4,5]).send(bob).send(alice)

In [23]:
bob._objects

{66639212157: [_LocalTensor - id:66639212157 owner:bob]}

In [24]:
alice._objects

{84419840883: [_PointerTensor - id:84419840883 owner:alice loc:bob id@loc:66639212157]}

In [25]:
x2 = x.end_get()

In [26]:
x2

FloatTensor[_PointerTensor - id:65137967680 owner:me loc:alice id@loc:84419840883]

In [27]:
bob._objects

{}

In [28]:
alice._objects

{84419840883: [_LocalTensor - id:84419840883 owner:alice]}

In [29]:
x2

FloatTensor[_PointerTensor - id:65137967680 owner:me loc:alice id@loc:84419840883]

In [30]:
x2.get()


 1
 2
 3
 4
 5
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

### Analyzing .end_get()

Notice above when we called .end_get(), it deleted bob's object and MOVED it to alice. So now Alice has the actual data (a LocalTensor). Thus, when we now call .get() on "x2" we will get the data back.

Now, you'll notice, before we called x2.get() we actually sent our tensor on a little journey. 

- First we sent the data to Bob. 
- Then we sent a pointer to the data to Alice
- Then we used Alice's pointer to MOVE the data to Alice (by calling .end_get())

Thus, we used this series of operations to MOVE the data from Bob -> Alice without us actually seeing the data during the in-between step. As you might guess - .move() is just a convenience wrapper around this operation!

In [31]:
# x is now a pointer to a pointer to the data which lives on Bob's machine
x = sy.FloatTensor([1,2,3,4,5]).send(bob)

In [32]:
bob._objects

{89478723216: [_LocalTensor - id:89478723216 owner:bob]}

In [33]:
alice._objects

{}

In [34]:
x = x.move(alice)

In [35]:
bob._objects

{}

In [36]:
alice._objects

{68996052490: [_LocalTensor - id:68996052490 owner:alice]}

In [37]:
x

FloatTensor[_PointerTensor - id:78674534083 owner:me loc:alice id@loc:68996052490]